This notebook replicates figures from Gao et al., 2018.

In [1]:
import datajoint as dj
from pipeline import ephys, behavior

Connecting root@db_gao2018:3306


In [2]:
cell_types = dj.U('unit_cell_type') & ephys.UnitSpikeTimes

In [3]:
key = ephys.UnitSpikeTimes().fetch('KEY')[0]

In [4]:
spike_trials = (ephys.UnitSpikeTimes & key).fetch1('spike_trials')
min_trial = min(spike_trials)
max_trial = max(spike_trials)
hit_r_trials = behavior.TrialSet.Trial & key & \
    'trial_response = "HitR"' & \
    'trial_id between {} and {}'.format(min_trial, max_trial)
err_r_trials = behavior.TrialSet.Trial & key & \
    'trial_response = "ErrR"' & \
    'trial_id between {} and {}'.format(min_trial, max_trial)
hit_l_trials = behavior.TrialSet.Trial & key & \
    'trial_response = "HitL"' & \
    'trial_id between {} and {}'.format(min_trial, max_trial)
err_l_trials = behavior.TrialSet.Trial & key & \
    'trial_response = "ErrL"' & \
    'trial_id between {} and {}'.format(min_trial, max_trial)

is_valid = len(hit_r_trials) > 15 & len(hit_l_trials) > 15

False

In [3]:
ephys.ValidUnit.delete()

About to delete:
`gao2018_ephys`.`__valid_unit`: 1923 items


Proceed? [yes, No]:  yes


Committed.
